# Exercise: Exploratory Data Analysis with Pandas
## Instructions
1. Data Collection
    - Search and collect data on the following: (1993 - 2023)
        - Israel interest tares
        - US interest rates
        - USD to ILS exchange rates
    - For each dataset, create a pandas DataFrame with the following columns:
        - Year
        - Month
        - Current_Rate (Interest rate or exchange rate for that month)
        - Change_From_Last_Month (Difference in Current_Rate from the previous month)
        - Optional: You may include additional columns if you find them useful (e.g., Change_Percentage, Date).



In [1]:
import pandas as pd
import numpy as np

In [10]:
# your code here
fed_rates_raw = pd.read_csv('./FEDFUNDS.csv')
fed_rates_raw = fed_rates_raw[(fed_rates_raw['DATE'] >= '1994-02-01') & (fed_rates_raw['DATE'] <= '2023-10-01')]
fed_rates_df = pd.DataFrame({
    'Year': fed_rates_raw['DATE'].str.split("-", expand=True)[0],
    'Month': fed_rates_raw['DATE'].str.split("-", expand=True)[1],
    'Current_Rate': fed_rates_raw['FEDFUNDS'],
    'Change_From_Last_Month': fed_rates_raw['FEDFUNDS'].diff()
})

israel_rates_raw = pd.read_csv('./BR.csv')
israel_rates_raw = israel_rates_raw[(israel_rates_raw['TIME_PERIOD'] >= '1994-02-01') & (israel_rates_raw['TIME_PERIOD'] <= '2023-10-01')]
israel_rates_df = pd.DataFrame({
    'Year': israel_rates_raw['TIME_PERIOD'].str.split("-", expand=True)[0],
    'Month': israel_rates_raw['TIME_PERIOD'].str.split("-", expand=True)[1],
    'Current_Rate': israel_rates_raw['OBS_VALUE'],
    'Change_From_Last_Month': israel_rates_raw['OBS_VALUE'].diff()
})


ils_usd_raw = pd.read_csv('./CCUSSP02ILM650N.csv')
ils_usd_raw = ils_usd_raw[(ils_usd_raw['DATE'] >= '1994-02-01') & (ils_usd_raw['DATE'] <= '2023-10-01')]
ils_usd_df = pd.DataFrame({
    'Year': ils_usd_raw['DATE'].str.split("-", expand=True)[0],
    'Month': ils_usd_raw['DATE'].str.split("-", expand=True)[1],
    'Current_Rate': ils_usd_raw['CCUSSP02ILM650N'],
    'Change_From_Last_Month': ils_usd_raw['CCUSSP02ILM650N'].diff()
})
israel_rates_df = israel_rates_df.drop_duplicates(subset=['Year', 'Month'], keep='first', inplace=False, ignore_index=False)
israel_rates_df

,Year,Month,Current_Rate,Change_From_Last_Month
5,1994,02,10.50,NaN
33,1994,03,10.50,0.0
64,1994,04,10.50,0.0
94,1994,05,10.50,0.0
125,1994,06,11.50,0.0
...,...,...,...,...
10717,2023,06,4.75,0.0
10747,2023,07,4.75,0.0
10778,2023,08,4.75,0.0
10809,2023,09,4.75,0.0


## Part A: Single DataFrame Analysis 
1. Display the first 5 rows of the US interest rates DataFrame.

2. Calculate the average Israeli interest rate over the period covered by your data.

3. Find the month and year when the USD to ILS exchange rate had the highest increase from the previous month.

4. Determine the total number of months where the US interest rate decreased compared to the previous month.

5. Add a new column to the Israeli interest rates DataFrame called Change_Percentage, representing the percentage change from the previous month.



In [3]:
# A - 1
# your code here
fed_rates_df.head(5)

,Year,Month,Current_Rate,Change_From_Last_Month
13,1994,02,3.25,NaN
14,1994,03,3.34,0.09
15,1994,04,3.56,0.22
16,1994,05,4.01,0.45
17,1994,06,4.25,0.24


In [8]:
# A - 2
# your code here
israel_rates_df['Current_Rate'].mean()

4.879243193354869

In [4]:
# A - 3
# your code here
ils_usd_df[['Year', 'Month']][ils_usd_df['Change_From_Last_Month'] == ils_usd_df['Change_From_Last_Month'].max()]

,Year,Month
465,1998,10


In [5]:
# A - 4
# your code here
fed_rates_df[fed_rates_df['Change_From_Last_Month'] < 0].shape[0]

119

In [6]:
# A - 5
# your code here
israel_rates_df['Change_Percentage'] = (israel_rates_df['Current_Rate'].pct_change() * 100).round(2)

## Part B: Merging DataFrames
1. Merge the US and Israeli interest rates DataFrames on Year and Month.

2. In the merged DataFrame, create a new column Rate_Difference that shows the difference between the Israeli and US interest rates for each month.

3. Identify all months when the Israeli interest rate was at least 1% higher than the US interest rate - IH (Israeli Higher).
Identify all months when the US interest rate was at least 1% higher than the Israeli interest rate - UH (US Higher).
Identify all months when the interest rates were within 1% of each other - N (Near)
make a new column called Rate_Comparison with the following values: IH, UH, or N.

4. Merge the exchange rate DataFrame with the merged interest rates DataFrame.

5. After merging all three DataFrames, find the correlation between the Rate_Difference and the Current_Rate of the exchange rate.
hint: find the connection between the difference in interest rates and the exchange rates, use any method you think is appropriate.
provide an explanation of the results (in Hebrew) - at least 2 rows of text + a proof of the connection(some output) you found.


In [11]:
# B - 1
# your code here

left = pd.DataFrame({'Year': israel_rates_df['Year'],
                     'Month': israel_rates_df['Month'],
                     'Israel_Rate': israel_rates_df['Current_Rate'],
                     'Israel_Rate_Difference': israel_rates_df['Change_From_Last_Month']
                    })

right = pd.DataFrame({'Year': fed_rates_df['Year'],
                     'Month': fed_rates_df['Month'],
                     'US_Rate': fed_rates_df['Current_Rate'],
                     'US_Rate_Difference': fed_rates_df['Change_From_Last_Month']})

merged_df = pd.merge(left,right, on=['Year','Month'])
fed_rates_df.head(10)
merged_df.head(10)


,Year,Month,Israel_Rate,Israel_Rate_Difference,US_Rate,US_Rate_Difference
0,1994,02,10.5,NaN,3.25,NaN
1,1994,03,10.5,0.0,3.34,0.09
2,1994,04,10.5,0.0,3.56,0.22
3,1994,05,10.5,0.0,4.01,0.45
4,1994,06,11.5,0.0,4.25,0.24
5,1994,07,12.0,0.0,4.26,0.01
6,1994,08,12.5,0.0,4.47,0.21
7,1994,09,14.0,1.5,4.73,0.26
8,1994,10,15.5,0.0,4.76,0.03
9,1994,11,15.5,0.0,5.29,0.53


In [20]:
# B - 2
# your code here
merged_df['Rate_Difference'] =  merged_df['Israel_Rate'] - merged_df['US_Rate']
merged_df

,Year,Month,Israel_Rate,Israel_Rate_Difference,US_Rate,US_Rate_Difference,percentage_diff,Rate_Comparison,Rate_Difference
0,1994,02,10.50,NaN,3.25,NaN,-69.047619,IH,7.25
1,1994,03,10.50,0.0,3.34,0.09,-68.190476,IH,7.16
2,1994,04,10.50,0.0,3.56,0.22,-66.095238,IH,6.94
3,1994,05,10.50,0.0,4.01,0.45,-61.809524,IH,6.49
4,1994,06,11.50,0.0,4.25,0.24,-63.043478,IH,7.25
...,...,...,...,...,...,...,...,...,...
352,2023,06,4.75,0.0,5.08,0.02,6.947368,UH,-0.33
353,2023,07,4.75,0.0,5.12,0.04,7.789474,UH,-0.37
354,2023,08,4.75,0.0,5.33,0.21,12.210526,UH,-0.58
355,2023,09,4.75,0.0,5.33,0.00,12.210526,UH,-0.58


In [13]:
# B - 3
# your code here
def percentage_change(col1,col2):
    return ((col2 - col1) / col1) * 100

merged_df['percentage_diff'] = percentage_change(merged_df['Israel_Rate'], merged_df['US_Rate'])

conditions = [
    (merged_df['percentage_diff'] < -1),
    (merged_df['percentage_diff'] > 1),
    ((merged_df['percentage_diff'] <= 1) & (merged_df['percentage_diff'] >= -1))
    ]

values = ['IH', 'UH', 'N']

merged_df['Rate_Comparison'] = np.select(conditions, values)


In [23]:
# B - 4
# your code here
ils_usd_to_merge = pd.DataFrame({'Year': ils_usd_df['Year'],
 'Month': ils_usd_df['Month'],
 'Ils_Usd': ils_usd_df['Current_Rate']})

fully_merged = pd.merge(merged_df, ils_usd_to_merge, on=['Year','Month'])
fully_merged

,Year,Month,Israel_Rate,Israel_Rate_Difference,US_Rate,US_Rate_Difference,percentage_diff,Rate_Comparison,Rate_Difference,Ils_Usd
0,1994,02,10.50,NaN,3.25,NaN,-69.047619,IH,7.25,2.975
1,1994,03,10.50,0.0,3.34,0.09,-68.190476,IH,7.16,2.969
2,1994,04,10.50,0.0,3.56,0.22,-66.095238,IH,6.94,3.000
3,1994,05,10.50,0.0,4.01,0.45,-61.809524,IH,6.49,3.037
4,1994,06,11.50,0.0,4.25,0.24,-63.043478,IH,7.25,3.033
...,...,...,...,...,...,...,...,...,...,...
352,2023,06,4.75,0.0,5.08,0.02,6.947368,UH,-0.33,3.700
353,2023,07,4.75,0.0,5.12,0.04,7.789474,UH,-0.37,3.693
354,2023,08,4.75,0.0,5.33,0.21,12.210526,UH,-0.58,3.801
355,2023,09,4.75,0.0,5.33,0.00,12.210526,UH,-0.58,3.824


In [26]:
# B - 5
# your code here
fully_merged['Rate_Difference'].corr(fully_merged['Ils_Usd'])


-0.10847248445996385

In [ ]:
זאת פונקציה שבודקת את הקשר בין המדדים והאם אחד עולה ויורד במקביל לשני.
אם יש תוצאה חיובית זה אומר ששער הדולר שקל עולה במקרה של ריבית יותר גבוהה בישראל
אם התוצאה שלילית לעומת זאת זה אומר ששער הדולר שקל יורד ככל שהריבית יותר גבוהה בישראל
התוצאה קרובה ל0 כלומר שיש מעט מאוד קורלציה

## Part C: Analyzing Relationships Between DataFrames
1. Analyze whether increases in US interest rates are associated with increases or decreases in the USD to ILS exchange rate in the same month.

2. Determine if changes in the Israeli interest rate have a stronger impact on the exchange rate than changes in the US interest rate.

3. Find any time lags in the effect of interest rate changes on the exchange rate. For example, does a change in US interest rates affect the exchange rate in the following month?
(כלומר- האם כשיש שינוי בריבית כלשהי - האם מייד יש שינוי בשער ההמרה? או שיש דיליי של חודש/חודשים עד שמרגישים את ההשפעה - אם בכלל?)

4. Calculate the rolling past 3-month average of the exchange rate for each data time, and add it as a new column.

5. Identify any periods where both US and Israeli interest rates were decreasing, and analyze how the exchange rate behaved during those periods.



In [27]:
fully_merged['US_Rate_Difference'].corr(fully_merged['Israel_Rate_Difference'])

0.11174167283174555

In [ ]:
# Bonus 

# Bonus
choose 3 global / local events or periods that you think could have affected the exchange rate or interest rates
and analyze the effect of these events on the data you collected.
show proof of your analysis that supports your claim.

(כלומר- תבחנו אירועים שקרו בתקופה שאתם עובדים עליה ובדקו איך הם השפיעו על הנתונים שאתם עובדים איתם)
למשל: בחירות בארה״ב, בחירות בישראל, רוסיה אוקראינה, מלחמות בישראל וכו׳.

In [ ]:
# Your code here